In [1]:
import pandas as pd
df = pd.read_csv('../../joyful-pandas-master/data/table.csv',index_col = 'ID')
df.head()

,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,
1101,S_1,C_1,M,street_1,173,63,34.0,A+
1102,S_1,C_1,F,street_2,192,73,32.5,B+
1103,S_1,C_1,M,street_2,186,82,87.2,B+
1104,S_1,C_1,F,street_2,167,81,80.4,B-
1105,S_1,C_1,F,street_4,159,64,84.8,B+


# 关于groupby

In [15]:
group = df.groupby('School')
print([g for g in dir(group) if not g.startswith('_')])

['Address', 'Class', 'Gender', 'Height', 'Math', 'Physics', 'School', 'Weight', 'agg', 'aggregate', 'all', 'any', 'apply', 'backfill', 'bfill', 'boxplot', 'corr', 'corrwith', 'count', 'cov', 'cumcount', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'dtypes', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'get_group', 'groups', 'head', 'hist', 'idxmax', 'idxmin', 'indices', 'last', 'mad', 'max', 'mean', 'median', 'min', 'ndim', 'ngroup', 'ngroups', 'nth', 'nunique', 'ohlc', 'pad', 'pct_change', 'pipe', 'plot', 'prod', 'quantile', 'rank', 'resample', 'rolling', 'sem', 'shift', 'size', 'skew', 'std', 'sum', 'tail', 'take', 'transform', 'tshift', 'var']


## 分组

### level参数用于多级索引，axis参数

In [8]:
df.set_index(['School','Class']).groupby(level = 1,axis = 0).first()

,Gender,Address,Height,Weight,Math,Physics
Class,,,,,,
C_1,M,street_1,173,63,34.0,A+
C_2,M,street_5,188,68,97.0,A-
C_3,M,street_4,161,68,31.5,B+
C_4,F,street_2,192,62,45.3,A


### 分组依据也可以是相同长度的列表

In [9]:
import numpy as np
df.groupby(np.random.choice(['a','b','c'],df.shape[0])).first()

,School,Class,Gender,Address,Height,Weight,Math,Physics
a,S_1,C_1,F,street_4,159,64,84.8,B+
b,S_1,C_1,M,street_2,186,82,87.2,B+
c,S_1,C_1,M,street_1,173,63,34.0,A+


### 支持函数型分组，传入的对象是索引

In [12]:
df[:5].groupby(lambda x:print(x)).head(0)

1101
1102
1103
1104
1105


,School,Class,Gender,Address,Height,Weight,Math,Physics
ID,,,,,,,,


#### a 根据奇偶行分组

In [17]:
df.groupby(lambda x:'奇数行' if df.index.get_loc(x) % 2 == 1 else '偶数行').groups

{'偶数行': Int64Index([1101, 1103, 1105, 1202, 1204, 1301, 1303, 1305, 2102, 2104, 2201,
             2203, 2205, 2302, 2304, 2401, 2403, 2405],
            dtype='int64', name='ID'),
 '奇数行': Int64Index([1102, 1104, 1201, 1203, 1205, 1302, 1304, 2101, 2103, 2105, 2202,
             2204, 2301, 2303, 2305, 2402, 2404],
            dtype='int64', name='ID')}

#### b 如果是多层索引，传入lambda的是元组

In [20]:
df.set_index(['School','Class']).head().groupby(lambda x : print(x)).head(0)

('S_1', 'C_1')
('S_1', 'C_1')
('S_1', 'C_1')
('S_1', 'C_1')
('S_1', 'C_1')


,,Gender,Address,Height,Weight,Math,Physics
School,Class,,,,,,


## 聚合

### agg函数的传入是分组逐列进行的

### 聚合可以指定哪些函数作用于哪些列

In [21]:
group.agg({'Math':['min','max'],'Height':'var'})

Math            Height
         min   max         var
School                        
S_1     31.5  97.0  161.638095
S_2     32.7  95.5  205.523684

### 可以用元组进行重命名

In [23]:
group['Math'].agg([('rename_mean','mean'),('rename_sum','sum')])

,rename_mean,rename_sum
School,,
S_1,63.746667,956.2
S_2,59.555000,1191.1


## 带参数的聚合函数

In [27]:
#判断是否组内数学分数至少有一个值在50-52之间
def f(x,low,high):
    return x.between(low,high).max()
group['Math'].agg(f,50,52)

School
S_1    False
S_2     True
Name: Math, dtype: bool

### 如果需要使用多个函数，并且其中至少有一个带参数，则使用wrap技巧

In [29]:
def f(x,low,high):
    return x.between(low,high).max()
def f_agg(f,name,*args,**kwargs):
    def wrapper(x):
        return f(x,*args,**kwargs)
    wrapper.__name__ = name
    return wrapper
new_f = f_agg(f,'score between 50 and 52',50,52)
group['Math'].agg([new_f,'mean'])

,score between 50 and 52,mean
School,,
S_1,False,63.746667
S_2,True,59.555000


## 过滤

### filter函数是用来筛选某些组的，筛选结果是组的全体，传入的值是布尔标量

In [38]:
group[['Math','Physics']].filter(lambda x:(x.Math > 32).all()).head()

,Math,Physics
ID,,
2101,83.3,C
2102,50.6,B+
2103,52.5,B-
2104,72.2,B+
2105,34.2,A


## 变换（transformation）

### transform函数传入的是组内的列，并且返回值需要与列长完全一致

In [40]:
group['Math'].transform(lambda x:x- x.min()).head()

ID
1101     2.5
1102     1.0
1103    55.7
1104    48.9
1105    53.3
Name: Math, dtype: float64

### 如果返回了标量值，那么组内所有的元素都会被广播为这个值

#### a 利用变换进行组内标准化

In [42]:
group[['Math','Height']].transform(lambda x:(x-x.mean())/x.std()).head()

,Math,Height
ID,,
1101,-1.288991,-0.214991
1102,-1.353990,1.279460
1103,1.016287,0.807528
1104,0.721627,-0.686923
1105,0.912289,-1.316166


#### b 利用变换进行组内缺失值的均值填充

In [47]:
df_nan = df[['Math','School']].copy().reset_index()
df_nan.loc[np.random.randint(0,df.shape[0],25),['Math']] = np.nan
df_nan.head()

,ID,Math,School
0,1101,34.0,S_1
1,1102,32.5,S_1
2,1103,87.2,S_1
3,1104,NaN,S_1
4,1105,84.8,S_1


In [50]:
df_nan.groupby('School').transform(lambda x:x.fillna(x.mean())).join(df.reset_index().School).head()

,ID,Math,School
0,1101,34.000000,S_1
1,1102,32.500000,S_1
2,1103,87.200000,S_1
3,1104,66.455556,S_1
4,1105,84.800000,S_1


## apply函数

### apply的传入值是分组的表

In [58]:
df.groupby('School').apply(lambda x:print(x.head(1)))

     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
1101    S_1   C_1      M  street_1     173      63  34.0      A+
     School Class Gender   Address  Height  Weight  Math Physics
ID                                                              
2101    S_2   C_1      M  street_7     174      84  83.3       C


""
